In [2]:
import pprint as pp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import xgboost as xgb
import copy as cp
import geoplot as gplt
import geopandas as gpd
%matplotlib inline
sns.set_style("white")

from geoplot import crs as gcrs
from shapely.geometry import Point, Polygon
from itertools import chain, product, cycle
from collections import Counter, OrderedDict
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from tap import modelmetrics, modelparams
from IPython.display import Image, display, HTML
from keras.utils import plot_model
from keras.models import load_model

D:\Anaconda3\envs\FYP_TAP_ENV\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
accidents = pd.read_csv("./pointer/acc2005_2016.csv")
accidents_processed = pd.read_csv("./pointer/acc2005_2016.proc.csv")
accidents_imputed = pd.read_csv("./Imputation/2.5/acc2005_2015-v2018.2.5.imp.csv")

D:\Anaconda3\envs\FYP_TAP_ENV\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
for col in accidents.columns[21:25]:
    print(col)

X2nd_Road_Number
Pedestrian_Crossing.Human_Control
Pedestrian_Crossing.Physical_Facilities
Light_Conditions


In [4]:
for col in accidents_processed.columns[15:19]:
    print(col)

X2nd_Road_Class
Pedestrian_Crossing_Human_Control
Pedestrian_Crossing_Physical_Facilities
Light_Conditions


In [14]:
accidents[accidents["Speed_limit"] == 0]

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing.Human_Control,Pedestrian_Crossing.Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
1761908,2015521507211,361824.0,173552.0,-2.550877,51.459648,52,3,2,1,13/08/2015,...,0,0,4,1,1,0,0,1,2,E01014681


In [15]:
accidents_processed[accidents_processed["Speed_Limit"] == 0]

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_Highway,...,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Date_Time,Is_Holiday,Year,Month,Hour


In [19]:
accidents.replace(-1, np.nan).isnull().sum()[20:25]

X2nd_Road_Class                            789860
X2nd_Road_Number                            17440
Pedestrian_Crossing.Human_Control             346
Pedestrian_Crossing.Physical_Facilities       795
Light_Conditions                               13
dtype: int64

In [14]:
accidents_processed.isnull().sum()[15:19]

X2nd_Road_Class                            783378
Pedestrian_Crossing_Human_Control             329
Pedestrian_Crossing_Physical_Facilities       499
Light_Conditions                               10
dtype: int64

In [17]:
accidents_imputed.isnull().sum()[12:14]

Pedestrian_Crossing_Physical_Facilities    0
Light_Conditions                           0
dtype: int64

In [18]:
accidents_imputed.merge(
    accidents[["Accident_Index", "Longitude", "Latitude"]], on="Accident_Index", 
    how="left").isnull().sum()[["Accident_Index", "Longitude", "Latitude", "Accident_Severity"]]

Accident_Index         0
Longitude            133
Latitude             133
Accident_Severity      0
dtype: int64